In [134]:
import weakref, sys, math
    
class GanttBase():
    def __init__(self, name):
        self.subitems = {}
        self.name = name
        
    def add_item(self, item):
        self.subitems[item.uid] = item
        
    def iter_all(self):
        for it in self.subitems.values():
            yield from it.iter_all()
            yield it
        
        
class GanttItem(GanttBase):
    def __init__(self, project, name, duration):
        super().__init__(name)
        self.duration = duration
        self.uid = project.allocate_uid()
        self.project = project
        self.descendents = weakref.WeakSet()
        self.precedents = weakref.WeakSet()
        
    def minstart(self):
        m = 0
        for p in self.precedents:
            mt = p.duration + p.minstart()
            if mt > m:
                m = mt
        return m
        
    def add_precedent(self, it):
        if isinstance(it, int):
            it = self.project.get_by_uid(it)
        self.precedents.add(it)
        
    def add_descendent(self, it):
        if isinstance(it, int):
            it = self.project.get_by_uid(it)
        self.descendents.add(it)
        
    def new_item(self, project, name, duration):
        new = self.__class__(project, name, duration)
        self.add_descendent(new)
        new.add_precedent(self)
        self.add_item(new)
        
    def iter_items(self):
        return iter(self.subitems.values())
        
    def free(self):
        for it in self.iter_items():
            it.free()
        self.subitems = []
        self.project = None    
        
    def __repr__(self):
        return "GanttItem(%r, %r, %r)"%(self.project, self.name, self.duration)
        
        
class GanttProject(GanttBase):
    def __init__(self, name):
        super().__init__(name)
        self._cur_id = 0
        
    def __repr__(self):
        return "GanttProject(%r)"%self.name
        
    def allocate_uid(self):
        rv = self._cur_id
        self._cur_id += 1
        return rv
    
    def get_by_uid(self, n):
        return self.subitems[n]
    
    def new_item(self, name, duration, prec=None):
        new = GanttItem(self, name, duration)
        if prec is not None:
            if isinstance(prec, int):
                prec = self.get_by_uid(prec)
            new.add_precedent(prec)
            prec.add_descendent(new)
        self.add_item(new)
        return new
    
    def print(self, fp=None):
        if fp is None:
            fp = sys.stdout
        
        tmpl = "%-40s |%s"
        
        out = []
        for it in self.iter_all():
            out.append((it.name, it.minstart(), it.duration))
            
        out.sort(key=lambda t: t[1])
        last = out[-1]
        end = last[1] + last[2]
        end = math.ceil(end/60)*60 
        
        nhours = math.ceil(end / 60)
        xaxis = []
#         for i in range(nhours):
#             s = str(i)
#             sl = len(s)
#             nleft = int((12 // 2) - (sl // 2))
#             nright = 12 - nleft - sl
#             xaxis.append("%s%s%s"%(nleft * " ", s, nright * " "))

            
        nhours = math.ceil(end / 60)
        nright = 11
        for i in range(nhours):
            s = str(i)
            sl = len(s)
            xaxis.append(" " * nright)
            xaxis.append(s)
            nright = int(10 - (sl // 2))
            
            
        sout = tmpl%("Activity", " ".join(xaxis))
        print(sout, file=fp)
        print("_"*len(sout), file=fp)
        
        for name, start, dur in out:
            sb = []
            i = 0
            dend = start + dur
            
            for j in range(int(end/5)+1):
                
                if i > 0 and not i % 60:
                    sb.append("|")
                if start <= (5*j) < dend:
                    sb.append("-")
                else:
                    sb.append(" ")
                i += 5
                
            padbar = "".join(sb)
            print(tmpl%(name, padbar), file=fp)
            

In [135]:
def add_to_proj(proj, qp, last=None):
    for name, dur in qp:
        new = proj.new_item(name, dur, last)
        last = new
    return last

proj = GanttProject("SemmaIQOQ")
last = add_to_proj(proj, iqp)
add_to_proj(proj, oqp, last)
with open("test.txt", 'w') as f:
    proj.print(f)

In [49]:
proj = GanttProject("SemmaIQOQ")
it = proj.new_item("Document Approvals", 10)
proj.new_item("Test Equipment", 10, it)
proj.print()

Document Approvals                       | --
Test Equipment                           |   --


In [31]:
from officelib.xllib import *
xl = Excel()

In [50]:
wb = xl.Workbooks("IOQ Time Estimate.xlsx")
ws = wb.Worksheets("IOQ")
cells = ws.Cells
cr = cells.Range

def data(c1):
    c2 = c1.End(xlc.xlDown).Offset(1, 6)
    return cr(c1, c2).Value

iq1 = cr("A3")
iq = data(iq1)

oq1 = cr("A27")
oq = data(oq1)

In [51]:
def parse(data):
    out = []
    for name, _, _, _, _, total in data:
        out.append((name, total / 60))
    return out

iqp = parse(iq)
oqp = parse(oq)

In [105]:
def add_to_proj(proj, qp, last=None):
    for name, dur in qp:
        new = proj.new_item(name, dur, last)
        last = new
    return last

proj = GanttProject("SemmaIQOQ")
last = add_to_proj(proj, iqp)
add_to_proj(proj, oqp, last)
with open("test.txt", 'w') as f:
    proj.print(f)

In [161]:
tasks = [
    ('Document Approvals', 10.0, False),
    ('Test Equipment', 10.0, False),
    ('System Accessories', 10.0, False),
    ('Product Info', 10.0, False),
    ('Physical inspection', 20.0, False),
    ('Plant utilities', 10.0, False),
    ('Gas Lines', 20.0, False),
    ('Bios Config / battery', 10.0, False),
    ('Power Up', 10.0, False),
    ('Software verif', 5.0, False),
    ('OS Config', 5.0, False),
    ('McAfee Config', 5.0, False),
    ('Inbound Firewall', 10.0, False),
    ('Outbound Firewall', 10.0, False),
    ('System Vars', 10.0, False),
    ('Alarms Off', 10.0, False),
    ('Alarms On', 10.0, False),
    ('Logging Off', 10.0, False),
    ('Logging On', 10.0, False),
    ('Email Settings', 5.0, False),
    ('Verify User Groups', 5.0, False),
    ('Protocol Completion', 10.0, False),
    ('Document Approvals', 10.0, False),
    ('Test Equipment', 30.0, False),
    ('Power On', 10.0, False),
    ('Vessel Fit', 10.0, False),
    ('Level Verification', 10.0, False),
    ('Agitation Control', 20.0, False),
    ('Transfer Config Files', 20.0, False),
    
    ('Temp Setup', 5, False),
    ('Temp Wait 1', 6.5*60, True),
    ('Temp Check', 25, False),
    ('Temp Wait 2', 4*60, True),
    ('Temp Check 2', 25, False),
#     ('Temp Control', 60.0, False),
    
    ('pH / DO Sensor Input', 5.0, False),
    
    ('pH Setup', 10, False),
    ('pH Wait 1', 60, True),
    ('pH Check', 5, False),
    
    ('DO Setup', 10, False),
    ('DO Wait 1', 40, True),
    ('DO Check 1', 5, False),
    ('DO Wait 2', 40, True),
    ('DO Check 2', 5, False),
    
#     ('Sensor Validation', 20.0, False),
    ('Gas Request', 5.0, False),
    ('Gas Flow Accuracy ', 15.0, False),
    ('Pumps', 5.0, False),
    
    ('Filter Oven Setup', 5, False),
    ('Filter Oven Wait', 60, True),
    ('Filter Oven Check', 5, False),
#     ('Filter Oven Temp', 5.0, False),
    
    ('Recipe Verification', 5.0, False),
    ('Power Loss & Restore', 15.0, False),
    ('Interlocks', 10.0, False),
    ('Process Alarms', 10.0, False),
    ('Clear Alarms', 5.0, False),
    ('Cleanup', 10.0, False),
    ('Flash Drive Verif', 10.0, False),
    ('Protocol Completion', 10.0, False),
]

In [240]:
import datetime

def fmt(dt):
    return dt.strftime("%I:%M %p")

def fmt2(dt):
    return dt.strftime("%m/%d/%y %I:%M:%S %p")

current = start

class Task():
    def __init__(self, br, name, dur, para, prec):
        self.br = br
        self.name = name
        self.dur = dur
        self.st = None
        self.prec = prec
        self.para = para
        self._done = False
        
    def running(self):
        return self.st is not None
    
    def finish(self):
        self._done = True
        
    def start(self, dt):
        self.st = dt
        
    def done(self, dt):
        if self._done:
            return True
        if not self.st:
            return False
        elapsed = self.elapsed(dt)
        return elapsed >= self.dur
        
    def elapsed(self, dt):
        return (dt - self.st).total_seconds() / 60 
    
    def __repr__(self):
        return "Task(%r, %r, Task<%s>)"%(self.name, self.dur, self.prec.name if self.prec else "Empty")
    

ioq_tasks = []

for br in range(4):
    br += 1
    prev = None
    for name, dur, para in tasks:
        task = Task(br, name, dur, para, prev)
        prev = task
        ioq_tasks.append(task)

start = startstart = current = datetime.datetime(2018, 12, 6, 9)
end_of_day = datetime.datetime(2018, 12, 6, 12+6)
oneday = datetime.timedelta(hours=24)
minute = datetime.timedelta(minutes=1)
brs = [
    0,
    0,
    0,
    0,
    0
]

out = []
def showtask(task, start, end):
    print("%d %-30s %30s %30s %s"%(task.br, task.name, fmt(start), fmt(end), "para" if task.para else ""))
    out.append((task.br, "<%d>%s"%(task.br, task.name), task.dur / 1440, fmt2(start)))

def elapsed():
    return (current - start).total_seconds() / 60

while ioq_tasks:
    el = elapsed()
    for task in ioq_tasks:
        if task.para and task.running() and task.done(current):
            ioq_tasks.remove(task)
            brs[task.br] = 0
            break
        
        prec = task.prec
        #import pdb;pdb.set_trace()
        if (prec is None or prec.done(current)) and brs[task.br] == 0:
            if task.para:
                task.start(current)
                end = current + datetime.timedelta(minutes=task.dur)
                showtask(task, current, end)
                brs[task.br] = 1
            else:
                if datetime.timedelta(minutes=task.dur) + current < end_of_day:
                    task.start(current)
                    end = current + datetime.timedelta(minutes=task.dur)
                    ioq_tasks.remove(task)
                    showtask(task, current, end)
                    current = end
                    break
    else:
        current += minute
        if current > end_of_day:
            start += oneday
            end_of_day += oneday
            current = start
                


#ioq_tasks

1 Document Approvals                                   09:00 AM                       09:10 AM 
1 Test Equipment                                       09:10 AM                       09:20 AM 
1 System Accessories                                   09:20 AM                       09:30 AM 
1 Product Info                                         09:30 AM                       09:40 AM 
1 Physical inspection                                  09:40 AM                       10:00 AM 
1 Plant utilities                                      10:00 AM                       10:10 AM 
1 Gas Lines                                            10:10 AM                       10:30 AM 
1 Bios Config / battery                                10:30 AM                       10:40 AM 
1 Power Up                                             10:40 AM                       10:50 AM 
1 Software verif                                       10:50 AM                       10:55 AM 
1 OS Config                             

In [241]:
def paste(values):
    c1 = xl.Selection
    c2 = c1.Offset(len(values), len(values[0]))
    cr = c1.Parent.Cells.Range
    cr(c1, c2).Value = values
paste(out)

In [199]:
def RGB(red, green, blue):
    return (red << 16) | (green << 8) | blue

(1, 'Document Approvals', 0.006944444444444444, '12/06/18 09:00:00 AM')

In [242]:
colors = {
    '1':(255,0,0),
    '2':(0,255,0),
    '3':(0,0,255),
    '4':(120,120,0),
}

ws = wb.Worksheets("Gantt2")
chart = ws.ChartObjects(ws.ChartObjects().Count).Chart
series = chart.SeriesCollection("Duration")
for x, p in zip(series.XValues, series.Points()):
    r,g,b = colors[x[1]]
    p.Interior.Color = RGB(r,g,b)